In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["HF_DATASETS_OFFLINE"] = "1"
import warnings
warnings.filterwarnings("ignore")
import json
from pathlib import Path
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

def save_merged_model(model, tokenizer, output_dir):
    """合并LoRA权重并保存完整模型"""
    # 合并LoRA适配器到基础模型
    if isinstance(model, PeftModel):
        merged_model = model.merge_and_unload()
    else:
        merged_model = model
    
    # 确保模型类型与原始配置一致
    original_config = merged_model.config.to_dict()
    if "model_type" not in original_config:
        original_config["model_type"] = "qwen2"  # 根据实际模型修改
    
    # 保存完整模型
    merged_model.save_pretrained(
        output_dir,
        safe_serialization=True,
        max_shard_size="5GB"  # 自动分片
    )
    
    # 显式保存修复后的配置文件
    with open(f"{output_dir}/config.json", "w") as f:
        json.dump(original_config, f, indent=4)
    
    # 保存分词器
    tokenizer.save_pretrained(output_dir)
    
    # 验证保存结果
    verify_saved_model(output_dir)

def verify_saved_model(model_path):
    """验证保存的模型完整性"""
    required_files = [
        "config.json",
        "model.safetensors",  # 或 pytorch_model.bin
        "tokenizer_config.json",
        "special_tokens_map.json",
        "tokenizer.model"  # 针对某些分词器的特殊文件
    ]
    
    print("\n保存文件结构验证:")
    for file in required_files:
        exists = (Path(model_path)/file).exists()
        print(f"{file}: {'✅' if exists else '❌'}")
    
    # 检查模型类型配置
    with open(f"{model_path}/config.json") as f:
        config = json.load(f)
        assert "model_type" in config, "缺少model_type字段"
        print(f"\n模型类型配置: {config['model_type']}")

# 使用示例
if __name__ == "__main__":
    # 加载原始模型
    base_model = AutoModelForCausalLM.from_pretrained(
        "./models/Qwen2.5-1.5B-Instruct",
        torch_dtype=torch.bfloat16,
        device_map="auto",
        use_cache=False
    )
    
    # 加载训练后的LoRA适配器
    trained_model = PeftModel.from_pretrained(
        base_model,
        "./grpo_finetuned_model",
        is_trainable=False
    )
    
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained(
        "./models/Qwen2.5-1.5B-Instruct"
    )
    
    # 保存合并后的完整模型
    save_merged_model(
        model=trained_model,
        tokenizer=tokenizer,
        output_dir="./models/merged_grpo_model"
    )



保存文件结构验证:
config.json: ✅
model.safetensors: ✅
tokenizer_config.json: ✅
special_tokens_map.json: ✅
tokenizer.model: ❌

模型类型配置: qwen2
